# MONTECARLO PARALELIZADO

## Instalaciones necesarias

- Para que el código a continuación funcione, es necesario tener instalado `dask`.

In [1]:
#%pip install dask   

- Para habilitar el dashboard, debemos installar `bokeh` :

In [2]:
#%pip install bokeh


In [3]:
#%%bash 

#sudo apt-get install -y graphviz

In [4]:
#%pip install -q graphviz

In [5]:
#import IPython
#app=IPython.Application.instance()
#app.kernel.do_shutdown(True) #restarting kernell

## Montecarlo 1D


Empezamos con el algoritmo de monte carlo en 1 dimensión

Integramos la función:
$$f(x)=\int_0^1 \frac{4}{1+x^2}=\pi$$ 




### Método secuencial:

In [6]:
import numpy as np
import math

In [7]:
#definimos la funcion del error relativo
err_rel=lambda ap,ob: math.fabs(ap-ob)/math.fabs(ob)

In [8]:
#numero de puntos a utilizar
density_p=10**8

#funcion a integrar
f=lambda x: 4/(1+x**2)

#resultado conocido de la integral (valor objetivo)
obj=math.pi

#intervalo
a=0
b=1

#dimensiones
dims=1

In [9]:
%%time
#approximación mediante método montecarlo
x_p=np.random.uniform(0,1,density_p) #obtiene tantos números aleatorios entre 0 y 1 como density_p.
vol=b-a # esta es la longitud del intervalo de integración.
approx=vol*np.mean(f(x_p))

print("---------------------")
print("El valor calculado de la integral es: ",approx)
print("El error relativo: {:0.4e}".format(err_rel(approx,obj)))
print("---------------------")

---------------------
El valor calculado de la integral es:  3.1415812651041706
El error relativo: 3.6251e-06
---------------------
CPU times: user 1.71 s, sys: 666 ms, total: 2.38 s
Wall time: 2.17 s


### Paralelización de MC en Dask

Para paralelizar estos resultados lo que pienso que tenemos que hacer es dividir las tareas en n subintervalos. La porpuesta es la siguiente:  
1. Dividir el dominio de la función en $p$ partes iguales. $p$ será el número de cpus disponibles en la máquina en que se corre la tarea.
2. Dividir el total de puntos a utilizar equitativamente entre $p$.
3. Resolver el método para cada intervalo.
4. Sumar los resultados de cada intervalo.

Nota: Para el caso de más dimensiones solamente se paraleliza una dimensión pues al hacerlo con todas se estaría dividiendo el espacio en $p_{dim}$ subespacios.

In [10]:
import multiprocessing
import time
from dask.distributed import Client
client = Client()

In [11]:
client

Client Scheduler: tcp://127.0.0.1:65426 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 17.18 GB


In [12]:
#paso 1. Dividir el dominio en partes iguales
p=multiprocessing.cpu_count() #cpus disponibles
n_subint=int(density_p/p) #numero de puntos o nodos en cada core o cpu

In [13]:
def construye_subintervalos1D(ids,a,b,p,dims):
    """

    Función que construye "p" subintervalos, delimitados entre a y b. El número de subintervalos \\
    dependerá del número de cores empleados. Este ejercicio fue ejecutado en una computadora \\
    con 16 cores, por lo tanto, se crean 16 subintervalos.
    
    Argumentos:
    ----------
    * ids: Identificador del core dónde se está corriendo el task. Este ejercicio fue ejecutado \\
           en una computadora con 16 cores, por ello los posibles valores de "ids" son números enteros \\
           entre 0 y 15.
    * a, b (float): Intervalo de integración [a, b]
    * p (int) : Número de cores o cpus disponibles
    * dims: Dimensión de la integral.
    
    Salidas:
    -------
    * (begin, end): Subintervalos de integración.
    
    Ejemplo:
    -------
    construye_subintervalos(ids=0,a,b,p,dims)
    return:
    (0.0, 0.0625)

    """
    tamano_int=(b-a)/p #tamaño de cada sub intervalo.
    begin= a + ids*tamano_int #construyen los subintervalo
    end=begin+tamano_int
    return (begin,end)

    
def evalua_subintervalos1D(intervalo,f,n_subint,dims):
    """
    Función que evalúa la integral en cada uno los p subintervalos \\
    previamente construidos con la función "construye_subintervalos"
    
    Argumentos:
    ----------
    * intervalo: Intervalo de la integral sobre la cual se va aproximar \\
                 la integral
    * f: Función que se está aproximando
    * n_subint: Número de puntos o nodos en cada core o cpu
    * dims: Dimensiones de la integral
    
    Salidas:
    --------
    + vol*np.mean(f(x_p)): Approximación a la integral
    
    """   
    x_p=np.random.uniform(intervalo[0],intervalo[1],n_subint) #draw samples from a uniform distribution
    vol=intervalo[1]-intervalo[0]
    return vol*np.mean(f(x_p)) #calcula la integral para una cada regió: subintervalo.

In [14]:
%%time

#submitting p function calls
futures_intervalos = client.map(construye_subintervalos1D,range(p),
                                                **{'a':a,
                                                   'b':b,
                                                   'p':p,
                                                   'dims':dims})

futures_evaluando=client.map(evalua_subintervalos1D, futures_intervalos,
                                             **{'f':f,
                                                'n_subint':n_subint,
                                                'dims':dims})

results=client.gather(futures_evaluando) # los resultados viven en varios workers 
aprox=sum(results) #sumamos los resultados de todos los workers

print("---------------------")
print("El valor calculado de la integral es: ",aprox)
print("El error relativo: {:0.4e}".format(err_rel(aprox,obj)))
print("---------------------")

---------------------
El valor calculado de la integral es:  3.1415945443255806
El error relativo: 6.0184e-07
---------------------
CPU times: user 99.2 ms, sys: 11 ms, total: 110 ms
Wall time: 564 ms


In [15]:
client.close()

Para el método en secuencial obtenemos un error relativo de $3.6251x10^{-6}$ y un tiempo de ejecución de $2.38$ seg y para el método en paralelo obtenemos un error relativo de $6.0184x10^{-7}$ en $110$ ms.